# Goodreads Book Reviews Analysis - Numerical Data Exploration

## Project Overview
This project aims to analyze **Goodreads book reviews**, focusing on **1-star ratings** to understand patterns in harsh reviews. The analysis is divided into two parts:
1. **Numerical Data Analysis** (Current Stage) - Examining numerical factors such as star ratings, review counts, and genre distributions.
2. **Natural Language Processing (NLP) Analysis** (Next Stage) - Exploring book descriptions and text reviews to identify sentiment patterns.

## Adding dataset with text reviews

In [ ]:
import pandas as pd
import json
import gzip

chunk_size= 10000
chunks= []

with gzip.open ("./Data/goodreads_reviews_dedup.json.gz", "rt", encoding="utf-8") as f:
    for i, line in enumerate(f): #read line by line
        chunks.append(json.loads(line)) #convert json to stionf dict

    #every chuck line, process data to write csv
        if (i + 1) % chunk_size == 0:
            df_chunk = pd.DataFrame(chunks)
            df_chunk.to_csv("goodreads_reviews", mode="a", index= False, header = (i < chunk_size))
            chunks = []
        
if chunks:
    df_chunk = pd.DataFrame(chunks)
    df_chunk.to_csv("goodreads_reviews", mode ="a", index=False, header=False) 


In [ ]:
df_reviews = pd.read_csv("goodreads_reviews")

In [ ]:
df_reviews.head()

In [ ]:
df_reviews.info()

In [ ]:
df_reviews['book_id'].duplicated().any()

In [ ]:
import pandas as pd
import json
import gzip

chunk_size= 10000
chunks= []

with gzip.open ("./Data/goodreads_books.json.gz", "rt", encoding="utf-8") as f:
    for i, line in enumerate(f): #read line by line
        chunks.append(json.loads(line)) #convert json to stionf dict
         
    #every chuck line, process data to write csv
        if (i + 1) % chunk_size == 0:
            df_chunk = pd.DataFrame(chunks)
            df_chunk.to_csv("goodreads_books", mode="a", index= False, header = (i < chunk_size))
            chunks = []
        
if chunks:
    df_chunk = pd.DataFrame(chunks)
    df_chunk.to_csv("goodreads_books", mode ="a", index=False, header=False) 

In [ ]:
df_books = pd.read_csv("goodreads_books")

In [ ]:
df_books.head(10)

In [ ]:
df_books.info()

In [ ]:
print(df_books.columns)

In [ ]:
df_merged = df_reviews.merge(df_books, on="book_id", how="inner")

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_merged.head(10)

In [ ]:
print(df_merged.columns)

In [ ]:
df_merged=df_merged.drop(columns=['user_id','date_added','read_at','started_at','date_updated','read_at','kindle_asin','work_id','n_comments','asin','similar_books','series','similar_books','publication_month','publication_day','edition_information','is_ebook'])


In [ ]:
df_merged.info()

In [ ]:
df_merged=df_merged.drop(columns=['format', 'num_pages', 'isbn13', 'link', 'title_without_series'])

In [ ]:
df_merged['review_id'].duplicated().any()

In [ ]:
(df_merged['text_reviews_count']== 0).any()

In [ ]:
df_merged[df_merged['text_reviews_count'] == 0]
#?? maybe outdated text review count

In [ ]:
df_merged[df_merged['rating'] == 0]
#reviews that have text but no star rating was left? I am choosing to leave these out of analysis

In [ ]:
df_merged= df_merged[df_merged['rating'].notna() & (df_merged['rating'] !=0)]

In [ ]:
#for this analysis I will only be focusing on english reviews
#removing nonenglish rows and rows with no text in review_text or description. I dont think this will hurt bc the df is so large
df_merged= df_merged.dropna(subset=['review_text','description'])

In [ ]:
df_merged.head()

In [ ]:
#cleaning popular shelves column
print(df_merged['popular_shelves'].iloc[0])

In [ ]:
#seeing which shelves have the highest counts
import ast
from collections import Counter

#function that extracts shelf names from string lists of the shelf dictionaires
def shelf_names(shelves_str):
    shelves_list = ast.literal_eval(shelves_str) #convert the string to a list of dicts
    if isinstance(shelves_list, list):
        return [shelf['name'] for shelf in shelves_list if 'name' in shelf] #extract 'name' value from each dict if it exists
    return []

shelf_counter = Counter()

In [ ]:
#very large operation (takes about 100 minutes to run)
for row in df_merged['popular_shelves'].dropna():
    shelf_counter.update(shelf_names(row))

print(shelf_counter.most_common(30))

In [ ]:
import random

unique_shelves = list(shelf_counter.keys())
print(f"unique names: {len(unique_shelves)}")

In [ ]:
print(shelf_counter.most_common(1000))

In [ ]:
def normalize_shelf(name):
    return name.strip().lower().replace(" ", "-")

In [ ]:
#Filtering shelf names

In [ ]:
#cleaning the author column
print(df_merged['authors'].iloc[0])

In [ ]:
#there is already a language code column but it's not through. Try lang detect to fill in missing
from langdetect import detect
df_merged['dec']

In [ ]:
#checking for final cleaning steps to slim down dataset futher before splitting  then saving to a csv

In [ ]:
#split df into managable chunks for further analysis

In [ ]:
for star in range(0,6):
    df_star = df_merged[df_merged['rating'] == star]
    df_star.to_csv(f"{star}star_reviews.csv")

In [ ]:
import zipfile
import os

csv_files = ["./Data/1star_reviews.csv"]

zip_path = "./Data/1star_reviews.zip"

with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for file in csv_files:
        arcname = os.path.basename(file)
        zipf.write(file,arcname=arcname)

zip_path

In [ ]:
#assigning them to variables then checking size

df_5star = pd.read_csv("./Data/5star_reviews.csv")
df_5star.info()

In [ ]:
df_4star = pd.read_csv("./Data/4star_reviews.csv")
df_4star.info()

In [ ]:
df_3star = pd.read_csv("./Data/3star_reviews.csv")
df_3star.info()

In [ ]:
df_2star = pd.read_csv("./Data/2star_reviews.csv")
df_2star.info()

In [1]:
import pandas as pd
import json
import gzip
import ast
from collections import Counter

In [ ]:
pip install "numpy<2"

In [3]:
df_1star = pd.read_csv("./1star_reviews.csv")
df_1star.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419874 entries, 0 to 419873
Data columns (total 20 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Unnamed: 0          419874 non-null  int64  
 1   book_id             419874 non-null  int64  
 2   review_id           419874 non-null  object 
 3   rating              419874 non-null  int64  
 4   review_text         419874 non-null  object 
 5   n_votes             419874 non-null  int64  
 6   isbn                328665 non-null  object 
 7   text_reviews_count  419874 non-null  float64
 8   country_code        419874 non-null  object 
 9   language_code       340979 non-null  object 
 10  popular_shelves     419874 non-null  object 
 11  average_rating      419874 non-null  float64
 12  description         419874 non-null  object 
 13  authors             419874 non-null  object 
 14  publisher           347484 non-null  object 
 15  publication_year    358879 non-nul

In [5]:
# taking a sample of the smallest rating dataset to test for cleaning
sample_1star= df_1star.sample(10000, random_state=42)

In [7]:
#cleaning popular shelves column
print(sample_1star['popular_shelves'].iloc[0])

[{'count': '587', 'name': 'to-read'}, {'count': '76', 'name': 'romance'}, {'count': '75', 'name': 'diana-palmer'}, {'count': '26', 'name': 'contemporary-romance'}, {'count': '20', 'name': 'currently-reading'}, {'count': '17', 'name': 'harlequin'}, {'count': '14', 'name': 'western'}, {'count': '13', 'name': 'contemporary'}, {'count': '10', 'name': 'long-tall-texans'}, {'count': '10', 'name': 'books-i-own'}, {'count': '9', 'name': 'fiction'}, {'count': '9', 'name': 'cowboy'}, {'count': '8', 'name': 'palmer-diana'}, {'count': '8', 'name': 'series'}, {'count': '7', 'name': 'palmer'}, {'count': '7', 'name': 'harlequin-romance'}, {'count': '6', 'name': 'western-romance'}, {'count': '5', 'name': 'owned'}, {'count': '5', 'name': 'default'}, {'count': '5', 'name': 'my-library'}, {'count': '4', 'name': 'kindle'}, {'count': '4', 'name': 'read-in-2010'}, {'count': '3', 'name': 'audio-books'}, {'count': '3', 'name': '2010-11'}, {'count': '3', 'name': 'harlequin-wishlist'}, {'count': '3', 'name': 'b

In [9]:
#seeing which shelves have the highest counts
#function that extracts shelf names from string lists of the shelf dictionaires
def shelf_names(shelves_str):
    shelves_list = ast.literal_eval(shelves_str) #convert the string to a list of dicts
    if isinstance(shelves_list, list):
        return [shelf['name'] for shelf in shelves_list if 'name' in shelf] #extract 'name' value from each dict if it exists
    return []

shelf_counter = Counter()

In [11]:
#very large operation (takes about 100 minutes to run)
for row in sample_1star['popular_shelves'].dropna():
    shelf_counter.update(shelf_names(row))

print(shelf_counter.most_common(60))

[('to-read', 9929), ('currently-reading', 9297), ('owned', 8457), ('fiction', 8311), ('favorites', 8263), ('books-i-own', 7961), ('kindle', 7382), ('ebook', 7213), ('library', 7085), ('owned-books', 6950), ('to-buy', 6593), ('ebooks', 6318), ('wish-list', 5941), ('default', 5726), ('contemporary', 5486), ('my-books', 5381), ('audiobook', 5368), ('adult', 5238), ('romance', 5224), ('audiobooks', 5077), ('i-own', 4903), ('my-library', 4853), ('did-not-finish', 4748), ('dnf', 4737), ('audio', 4647), ('abandoned', 4567), ('favourites', 4412), ('e-book', 4404), ('series', 4208), ('novels', 4206), ('read-in-2015', 4022), ('own-it', 3976), ('books', 3873), ('book-club', 3829), ('fantasy', 3827), ('e-books', 3779), ('read-in-2016', 3764), ('read-in-2014', 3740), ('adult-fiction', 3735), ('maybe', 3717), ('young-adult', 3559), ('read-in-2013', 3381), ('read-in-2017', 3149), ('mystery', 3113), ('have', 3004), ('novel', 2992), ('reviewed', 2966), ('borrowed', 2906), ('ya', 2884), ('audible', 2854

In [13]:
import random

unique_shelves = list(shelf_counter.keys())
print(f"unique names: {len(unique_shelves)}")

unique names: 92620


In [37]:
import ast

blacklist = {
    'reading_status': [
        'read', 'currently reading', 'dnf', 'unread', 'tbr', 'reread', 'finished'
    ],
    'ownership': [
        'owned', 'own', 'buy', 'bought', 'borrow', 'library', 'kindle', 'ebook', 'epub', 
        'paperback', 'nook', 'hardcover', 'download', 'ibooks', 'kobo', 'scribd'
    ],
    'rating_review': [
        'star', 'favorite', 'favourite', 'review', 'recommend', 'amazing', 'must', 
        'best', 'loved'
    ],
    'promotion': [
        'audiobook', 'audio', 'netgalley', 'gift', 'challenge', 'award', 'edition', 
        'collection', 'release', 'published', 'sequel', 'shelve', 'scan', 'pdf'
    ],
    'proper_nouns': [
        'neal', 'stephenson', 'amy', 'kate', 'robert', 'emily', 'veronica', 'june', 'sophia', 'palmer'
    ],
    'misc': [
        'storage', 'location', 'page', 'purchase', 'bore', 'new', 'hold', 'mine', 
        'drop', 'theme', 'funny', 'didnt', 'purchased', 'print'
    ]
}


# Flatten into a set of lowercase blacklist words
blacklist_set = set()
for group in blacklist.values():
    blacklist_set.update(word.lower() for word in group)

In [17]:
import ast

def clean_name(name):
    return name.lower().replace('-', ' ').replace('_', ' ').strip()

def extract_shelves(shelves_str):
    try:
        shelves_list = ast.literal_eval(shelves_str)
    except:
        return []

    if isinstance(shelves_list, list):
        return [(clean_name(shelf['name']), int(shelf.get('count', 0)))
                for shelf in shelves_list if 'name' in shelf]
    return []

In [19]:
sample_1star['cleaned_shelves'] = sample_1star['popular_shelves'].apply(extract_shelves)

In [23]:
pd.set_option('display.max_colwidth', None)
sample_1star[['cleaned_shelves']].head(20)

,cleaned_shelves
41230,"[(to read, 587), (romance, 76), (diana palmer, 75), (contemporary romance, 26), (currently reading, 20), (harlequin, 17), (western, 14), (contemporary, 13), (long tall texans, 10), (books i own, 10), (fiction, 9), (cowboy, 9), (palmer diana, 8), (series, 8), (palmer, 7), (harlequin romance, 7), (western romance, 6), (owned, 5), (default, 5), (my library, 5), (kindle, 4), (read in 2010, 4), (audio books, 3), (2010 11, 3), (harlequin wishlist, 3), (books i have, 3), (adult, 3), (cowboys, 3), (favorites, 3), (virgin heroine, 3), (april, 3), (harlequim, 2), (storage1, 2), (did not finish, 2), (series in progress, 2), (150 250 pages, 2), (chick lit, 2), (books, 2), (on my shelf, 2), (adult fiction, 2), (cowboy western, 2), (terjemahan, 2), (genre western, 2), (undecided, 2), (paper back, 2), (not interested, 2), (gramedia, 2), (favourites, 2), (bought, 2), (romantic suspense, 2), (books owned, 2), (action, 2), (library, 2), (i own, 2), (small town, 2), (audiobook, 2), (part of a series, 2), (series romance, 2), (library books, 2), (2010 read, 2), (harlequinromance, 2), (2017 read, 1), (meaghan, 1), (mills and boon, 1), (my owned books, 1), (elibrary, 1), (0 contemporary, 1), (genre romance, 1), (seasonal challenge, 1), (best covers, 1), (read in 2015, 1), (meh, 1), (all time favorites, 1), (tbr later, 1), (harlequins, 1), (books i own paper, 1), (angsty, 1), (on the shelf, 1), (2017 challenge, 1), (series read, 1), (free borrowed, 1), (paperback, 1), (shelf 4 front, 1), (2017 bookriot challenge, 1), (vicky to read, 1), (need to read vet, 1), (1999 reads, 1), (2010 reads, 1), (blom check, 1), (primary, 1), (romance modern, 1), (own to read, 1), (long tall texan, 1), (location nook, 1), (hero grovels, 1), (brooding hero, 1), (long tall texans series bk 34, 1), (harlequin modern romance, 1), (e books, 1), (3 stars, 1)]"
95710,"[(to read, 25568), (science fiction, 1975), (sci fi, 1674), (currently reading, 1113), (fiction, 1014), (scifi, 453), (favorites, 344), (post apocalyptic, 211), (sf, 179), (kindle, 175), (audiobook, 170), (owned, 159), (sci fi fantasy, 138), (abandoned, 128), (ebook, 125), (audiobooks, 122), (dystopian, 114), (audible, 111), (library, 111), (audio, 106), (space, 105), (to buy, 101), (fantasy, 100), (dystopia, 95), (apocalyptic, 93), (read in 2015, 81), (read in 2016, 79), (science, 73), (speculative fiction, 70), (scifi fantasy, 68), (dnf, 66), (novels, 63), (did not finish, 63), (fantasy sci fi, 58), (ebooks, 57), (adult, 57), (to read fiction, 57), (apocalypse, 57), (books i own, 54), (space opera, 51), (read 2015, 49), (wish list, 48), (unfinished, 47), (read 2016, 46), (book club, 46), (bill gates, 44), (didn t finish, 43), (science fiction fantasy, 42), (hard sci fi, 38), (audio book, 37), (audio books, 36), (maybe, 36), (sff, 34), (adventure, 32), (gave up on, 31), (on hold, 31), (novel, 31), (recommended, 31), (read 2017, 30), (read in 2017, 30), (owned books, 30), (e books, 29), (post apocalypse, 29), (sf fantasy, 28), (e book, 28), (survival, 28), (default, 27), (neal stephenson, 27), (to read sci fi, 27), (sci fi and fantasy, 26), (never finished, 24), (sci fi to read, 24), (hard scifi, 23), (couldn t finish, 23), (to read scifi, 23), (epic, 23), (stopped reading, 22), (hard science fiction, 22), (to get, 22), (literature, 21), (2015 reads, 21), (adult fiction, 21), (standalone, 20), (fiction to read, 20), (fantasy scifi, 20), (english, 20), (sciencefiction, 20), (speculative, 20), (thriller, 19), (borrowed, 18), (library books, 18), (gave up, 18), (signed, 18), (future, 18), (reviewed, 17), (favourites, 17), (2015 read, 17), (my library, 17), (21st century, 17), (near future, 17)]"
142885,"[(currently reading, 247), (romance, 150), (to read, 111), (kindle, 97), (historical romance, 80), (historical, 78), (historical fiction, 53), (regency, 53), (ebook, 38), (humor, 30), (fiction, 19), (ebooks, 18), (owned, 17), (kindle books, 17), (favorites, 15), (regency romance, 14), (e book

In [25]:
def remove_blacklisted_shelves(cleaned_shelves, blacklist_words):
    """
    Removes shelves that match any blacklist word.
    
    cleaned_shelves: list of (name, count) tuples [(shelf1, count1), (shelf2, count2), ...]
    blacklist_words: set of lowercase blacklist words
    """
    result = []
    for name, count in cleaned_shelves:
        # If none of the blacklist words are contained in the name, keep it
        if not any(bad_word in name for bad_word in blacklist_words):
            result.append((name, count))
    return result

In [33]:
sample_1star['cleaned_shelves'] = sample_1star['cleaned_shelves'].apply(lambda shelves: remove_blacklisted_shelves(shelves, blacklist_words))

In [34]:
sample_1star[['cleaned_shelves']].head(20)

,cleaned_shelves
41230,"[(romance, 76), (contemporary romance, 26), (harlequin, 17), (western, 14), (contemporary, 13), (long tall texans, 10), (fiction, 9), (cowboy, 9), (series, 8), (harlequin romance, 7), (western romance, 6), (default, 5), (2010 11, 3), (harlequin wishlist, 3), (books i have, 3), (adult, 3), (cowboys, 3), (virgin heroine, 3), (april, 3), (harlequim, 2), (did not finish, 2), (series in progress, 2), (chick lit, 2), (books, 2), (on my shelf, 2), (adult fiction, 2), (cowboy western, 2), (terjemahan, 2), (genre western, 2), (undecided, 2), (paper back, 2), (not interested, 2), (gramedia, 2), (romantic suspense, 2), (action, 2), (part of a series, 2), (series romance, 2), (harlequinromance, 2), (meaghan, 1), (mills and boon, 1), (0 contemporary, 1), (genre romance, 1), (meh, 1), (harlequins, 1), (angsty, 1), (on the shelf, 1), (shelf 4 front, 1), (blom check, 1), (primary, 1), (romance modern, 1), (long tall texan, 1), (hero grovels, 1), (brooding hero, 1), (long tall texans series bk 34, 1), (harlequin modern romance, 1), (e books, 1)]"
95710,"[(science fiction, 1975), (sci fi, 1674), (fiction, 1014), (scifi, 453), (post apocalyptic, 211), (sf, 179), (sci fi fantasy, 138), (abandoned, 128), (dystopian, 114), (audible, 111), (space, 105), (fantasy, 100), (dystopia, 95), (apocalyptic, 93), (science, 73), (speculative fiction, 70), (scifi fantasy, 68), (novels, 63), (did not finish, 63), (fantasy sci fi, 58), (adult, 57), (apocalypse, 57), (space opera, 51), (wish list, 48), (book club, 46), (bill gates, 44), (didn t finish, 43), (science fiction fantasy, 42), (hard sci fi, 38), (maybe, 36), (sff, 34), (adventure, 32), (gave up on, 31), (novel, 31), (e books, 29), (post apocalypse, 29), (sf fantasy, 28), (e book, 28), (survival, 28), (default, 27), (sci fi and fantasy, 26), (hard scifi, 23), (couldn t finish, 23), (epic, 23), (hard science fiction, 22), (to get, 22), (literature, 21), (adult fiction, 21), (standalone, 20), (fantasy scifi, 20), (english, 20), (sciencefiction, 20), (speculative, 20), (thriller, 19), (gave up, 18), (signed, 18), (future, 18), (21st century, 17), (near future, 17)]"
142885,"[(romance, 150), (historical romance, 80), (historical, 78), (historical fiction, 53), (regency, 53), (humor, 30), (fiction, 19), (regency romance, 14), (e book, 13), (comedy, 11), (romance historical, 10), (free, 9), (adult, 8), (freebie, 8), (freebies, 7), (chick lit, 6), (e books, 6), (amazon, 5), (england, 4), (wish list, 4), (cross dressing, 4), (guilty pleasures, 4), (d l carter, 3), (humorous, 3), (free books, 3), (maybe, 3), (gender bender, 3), (regency era, 3), (fantasy, 3), (amazon freebies, 3), (humour, 3), (historicals, 3), (friendship, 2), (default, 2), (laugh out loud, 2), (shelfari humor, 2), (humour comedy, 2), (humorous romance, 2), (period, 2), (m f, 2), (friends to lovers, 2), (bookbub, 2), (fun, 2), (misc, 2), (mf, 2), (fiction humor, 2), (historical romances, 2), (drama, 2), (family, 2), (love story, 2)]"
238919,"[(young adult, 640), (dystopian, 421), (dystopia, 345), (ya, 335), (science fiction, 222), (post apocalyptic, 186), (series, 182), (sci fi, 157), (survival, 120), (fiction, 109), (apocalyptic, 101), (teen, 71), (adventure, 61), (wish list, 52), (apocalypse, 51), (fantasy, 48), (arc, 42), (maybe, 39), (ya fiction, 39), (monument 14, 36), (signed, 33), (end of the world, 32), (first in series, 29), (ya dystopian, 27), (realistic fiction, 27), (my books, 25), (scifi, 24), (romance, 23), (ya books, 23), (young adult fiction, 21), (dystopian post apocalyptic, 21), (teen fiction, 20), (suspense, 20), (did not finish, 19), (post apocalypse, 19), (male pov, 19), (thriller, 18), (horror, 17), (action adventure, 17), (high school, 15), (sci fi fantasy, 15), (disaster, 15), (speculative fiction, 14), (future, 14), (ya lit, 14), (want, 14), (contemporary, 13), (have, 13), (action, 13), (meh, 12), (ya dystopia, 12), (distopian, 12), (book club, 12), (friendship, 12), (first in a series, 12), (colo

In [ ]:
import numpy as np
import html
import re

In [ ]:
#attempting to clean the html first
